In [50]:
#https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
%load_ext autoreload
%autoreload 2

import json
from datetime import datetime
#from Kafka
from OpenWeatherMap import OpenWeatherMap

In [53]:
from gc import collect


openWeatherMap = OpenWeatherMap()
#kafka = KafkaWriter()


def format_date(dt: str) -> str:
    return datetime.fromtimestamp(float(dt)).strftime('%d.%m.%Y %H:%M')


# load 'locations.json' into a json-object and return it
def load_locations() -> json:
    with open('locations.json', mode='r') as file:
        return json.load(file)


# for each location, query openWeatherMap for the 5-day forecast and 
# store the the returned values in Kafka
def collect_forecast_data() -> None:
    cities = load_locations()
    for key in cities:
         city = cities[key]
         print(openWeatherMap.get_forecast(city))

#enter your code

print(load_locations())
collect_forecast_data()

{'Bregenz': {'latitude': 47.5025779, 'longitude': 9.7472924}, 'Ravensburg': {'latitude': 47.7811014, 'longitude': 9.612468}}
{'coord': {'lon': 9.7473, 'lat': 47.5026}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'Ein paar Wolken', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 25.35, 'feels_like': 25.48, 'temp_min': 24.01, 'temp_max': 26.05, 'pressure': 989, 'humidity': 59}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 280}, 'clouds': {'all': 20}, 'dt': 1661099175, 'sys': {'type': 2, 'id': 2044490, 'country': 'AT', 'sunrise': 1661055893, 'sunset': 1661106234}, 'timezone': 7200, 'id': 2781503, 'name': 'Bregenz', 'cod': 200}
{'coord': {'lon': 9.6125, 'lat': 47.7811}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'Ein paar Wolken', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 24.58, 'feels_like': 24.22, 'temp_min': 23.68, 'temp_max': 25.72, 'pressure': 1015, 'humidity': 43}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 100}, 'clouds': {'